<a href="https://colab.research.google.com/github/RodionHorbunov/Algorithms-for-massive-data/blob/main/Finding_similar_items_Rodion_Horbunov_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title IMPORTING LIBRARIES
import zipfile
import pandas as pd
import numpy as np
import glob
import itertools


import string
import nltk
nltk.download('punkt_tab')
nltk.download('wordnet')
from nltk.tokenize import (sent_tokenize, word_tokenize)
from nltk import ngrams
#from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import regexp_tokenize
#Stop words
#from nltk.corpus import stopwords
#Lemmatizer
#from nltk.stem import WordNetLemmatizer
!pip install unidecode
import unidecode

#Stemmer
#from nltk.stem import PorterStemmer

import re
import os
import sys
import time
import tracemalloc
from multiprocessing import Pool, cpu_count
from IPython.display import clear_output
# Clear the output
clear_output()

!pip install datasketch
from datasketch import MinHash, MinHashLSH


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 kB 2.5 MB/s eta 0:00:00


In [2]:
#@title Importing credentials for Kaggle
os.environ['KAGGLE_USERNAME'] = "rodionhorbunov"
os.environ['KAGGLE_KEY'] = "e37ca7b26f7537fabcdf49a42416df7a"
!kaggle datasets download -d asaniczka/1-3m-linkedin-jobs-and-skills-2024

clear_output()

In [3]:
#@title Downloading dataset, unzip and extract all files, delete useless one

# Set the zip path
zip_path = '/content/1-3m-linkedin-jobs-and-skills-2024.zip'

# Specify the directory to extract the contents of the ZIP file to
extract_path = '/content/linkedin-jobs-and-skills'

# Open the ZIP file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    # Extract all the contents of the ZIP file to the extract_path directory
    zip_ref.extractall(extract_path)


In [4]:
#@title Downloading dataset, unzip and extract all files
df = pd.read_csv('/content/linkedin-jobs-and-skills/job_summary.csv', nrows=100000)

print(df.head(5))

# Check the duplicates
df_len = len(df)
df_len_wo_dupl = len(df.drop_duplicates())

print("Lenght - ",df_len, "Length w/o duplicates",df_len_wo_dupl,"\n")
#Almost 50 sec for this operations

#The same position and different linkdin posts

unique_positions = len(df["job_summary"].drop_duplicates())
print("There are",unique_positions," same position with different linkdin posts","\n")

#Identical positions (different post time)
duplicates_position = df_len - unique_positions
print("There are",duplicates_position,"identical positions","\n")

#Share of identical position
print("Share of identical position", (1297332-958192)/1297332,"\n")



                                            job_link  \
0  https://www.linkedin.com/jobs/view/restaurant-...   
1  https://www.linkedin.com/jobs/view/med-surg-re...   
2  https://www.linkedin.com/jobs/view/registered-...   
3  https://uk.linkedin.com/jobs/view/commercial-a...   
4  https://www.linkedin.com/jobs/view/store-manag...   

                                         job_summary  
0  Rock N Roll Sushi is hiring a Restaurant Manag...  
1  Schedule\n: PRN is required minimum 12 hours p...  
2  Description\nIntroduction\nAre you looking for...  
3  Commercial account executive\nSheffield\nFull ...  
4  Address:\nUSA-CT-Newington-44 Fenn Road\nStore...  
Lenght -  100000 Length w/o duplicates 100000 

There are 79037  same position with different linkdin posts 

There are 20963 identical positions 

Share of identical position 0.2614134238575785 



In [5]:
#@title  Sampling 10%, Fixing the indexes, mapping the dataset and providing the grouping (cleaning from duplicates)

df = df.reset_index()

result_df = df.groupby('job_summary').agg(
    min_index=('index', lambda x: x.index.min()),
    all_indexes=('index', lambda x: list(x.index))
)

In [6]:
#@title Delete downloaded dataset
del df
del df_len
del df_len_wo_dupl
del unique_positions
del duplicates_position
import gc
gc.collect()

68

In [7]:
#@title Processing new dataset
result_df = result_df.reset_index()
result_df["nbr_of_same_jobs"] = result_df["all_indexes"].apply(lambda x: len(x))

In [25]:
#@title Creating test sample of 10k rows
test_df = result_df[0:10000]

In [26]:
#@title Processing test dataset with straight forward computing
k = 5
threshold_similarity = 0.6

data =  test_df['job_summary']
# Create an MinHashLSH index optimized for Jaccard threshold 0.5,
# that accepts MinHash objects with 128 permutations functions
lsh = MinHashLSH(threshold=threshold_similarity, num_perm=32)

# Create MinHash objects
minhashes = {}
for c, i in enumerate(data):
  minhash = MinHash(num_perm=32)
  word_tokenized = word_tokenize(re.sub(r"[^\w\s]", " ", i.lower()), language="english", preserve_line=False)
  for j in range(len(word_tokenized) - k + 1):
    minhash.update(" ".join(word_tokenized[j:j + k]).encode('utf-8'))
  lsh.insert(test_df.loc[c,'min_index'], minhash)
  minhashes[test_df.loc[c,'min_index']] = minhash


#for i in range(len(minhashes.keys())):
#  result = lsh.query(minhashes[i])
#  print(f"Candidates with Jaccard similarity > {threshold_similarity} for input", i, ":", result)

In [27]:
#@title Check the results
index_to_check = 35065

same_job_descriptions = set(lsh.query(minhashes[index_to_check]) +result_df.loc[index_to_check,'all_indexes'])

print(same_job_descriptions)

for index in same_job_descriptions:
  print(result_df[result_df['min_index']==index]['job_summary'])


{40736, 25473, 35065, 1131, 40950, 41689}
32    "Is your passion in retail?” We are looking fo...
Name: job_summary, dtype: object
33    "Is your passion in retail?” We are looking fo...
Name: job_summary, dtype: object
30    "Is your passion in retail?” We are looking fo...
Name: job_summary, dtype: object
35065    Job Description:\nKitchen Manager\nEmployer:\n...
Name: job_summary, dtype: object
29    "Is your passion in retail?” We are looking fo...
Name: job_summary, dtype: object
31    "Is your passion in retail?” We are looking fo...
Name: job_summary, dtype: object


In [28]:
#@title Processing test dataset with parallel computing

data =  test_df['job_summary']
k = 5
threshold_similarity = 0.6


# Define a function for processing each item
def process_item(args):
    c, i, result_row, k = args
    minhash = MinHash(num_perm=32)
    word_tokenized = word_tokenize(re.sub(r"[^\w\s]", " ", i.lower()), language="english", preserve_line=False)
    for j in range(len(word_tokenized) - k + 1):
        minhash.update(" ".join(word_tokenized[j:j + k]).encode('utf-8'))
    return result_row['min_index'], minhash

# Initialize LSH and MinHashes
minhashes = {}
lsh = MinHashLSH(threshold=threshold_similarity, num_perm=32)

# Set your desired value of k
data_with_indices = [(c, i, test_df.loc[c], k) for c, i in enumerate(data)]

# Use multiprocessing to process the data
with Pool(processes=cpu_count()) as pool:
    results = pool.map(process_item, data_with_indices)

# Update LSH and MinHashes after parallel processing
for min_index, minhash in results:
    lsh.insert(min_index, minhash)
    minhashes[min_index] = minhash


In [29]:
#@title Check the results
index_to_check = 35065

same_job_descriptions = set(lsh.query(minhashes[index_to_check]) +result_df.loc[index_to_check,'all_indexes'])

print(same_job_descriptions)

for index in same_job_descriptions:
  print(result_df[result_df['min_index']==index]['job_summary'])


{40736, 25473, 35065, 1131, 40950, 41689}
32    "Is your passion in retail?” We are looking fo...
Name: job_summary, dtype: object
33    "Is your passion in retail?” We are looking fo...
Name: job_summary, dtype: object
30    "Is your passion in retail?” We are looking fo...
Name: job_summary, dtype: object
35065    Job Description:\nKitchen Manager\nEmployer:\n...
Name: job_summary, dtype: object
29    "Is your passion in retail?” We are looking fo...
Name: job_summary, dtype: object
31    "Is your passion in retail?” We are looking fo...
Name: job_summary, dtype: object


In [30]:
#@title Check memory usage of all variables
for var_name, var_obj in sorted(vars().items()):
  print(f"{var_name}: {sys.getsizeof(var_obj)/1024**2:.2f} MB")

In: 0.00 MB
MinHash: 0.00 MB
MinHashLSH: 0.00 MB
Out: 0.00 MB
Pool: 0.00 MB
_: 0.00 MB
_12: 0.00 MB
_13: 0.03 MB
_15: 0.00 MB
_16: 0.00 MB
_17: 0.00 MB
_18: 0.27 MB
_20: 0.00 MB
_21: 0.00 MB
_22: 0.00 MB
_23: 0.00 MB
_6: 0.00 MB
__: 0.00 MB
___: 0.00 MB
__builtin__: 0.00 MB
__builtins__: 0.00 MB
__doc__: 0.00 MB
__loader__: 0.00 MB
__name__: 0.00 MB
__package__: 0.00 MB
__spec__: 0.00 MB
_dh: 0.00 MB
_exit_code: 0.00 MB
_i: 0.00 MB
_i1: 0.00 MB
_i10: 0.00 MB
_i11: 0.00 MB
_i12: 0.00 MB
_i13: 0.00 MB
_i14: 0.00 MB
_i15: 0.00 MB
_i16: 0.00 MB
_i17: 0.00 MB
_i18: 0.00 MB
_i19: 0.00 MB
_i2: 0.00 MB
_i20: 0.00 MB
_i21: 0.00 MB
_i22: 0.00 MB
_i23: 0.00 MB
_i24: 0.00 MB
_i25: 0.00 MB
_i26: 0.00 MB
_i27: 0.00 MB
_i28: 0.00 MB
_i29: 0.00 MB
_i3: 0.00 MB
_i30: 0.00 MB
_i4: 0.00 MB
_i5: 0.00 MB
_i6: 0.00 MB
_i7: 0.00 MB
_i8: 0.00 MB
_i9: 0.00 MB
_ih: 0.00 MB
_ii: 0.00 MB
_iii: 0.00 MB
_oh: 0.00 MB
c: 0.00 MB
clear_output: 0.00 MB
cpu_count: 0.00 MB
data: 83.36 MB
data_with_indices: 0.08 MB
exit: 

In [31]:
del test_df
gc.collect()

0

In [32]:
#@title Processing full dataset
data =  result_df['job_summary']
k = 5
threshold_similarity = 0.6


# Define a function for processing each item
def process_item(args):
    c, i, result_row, k = args
    minhash = MinHash(num_perm=32)
    word_tokenized = word_tokenize(re.sub(r"[^\w\s]", " ", i.lower()), language="english", preserve_line=False)
    for j in range(len(word_tokenized) - k + 1):
        minhash.update(" ".join(word_tokenized[j:j + k]).encode('utf-8'))
    return result_row['min_index'], minhash

# Initialize LSH and MinHashes
minhashes = {}
lsh = MinHashLSH(threshold=threshold_similarity, num_perm=32)

data_with_indices = [(c, i, result_df.loc[c], k) for c, i in enumerate(data)]

# Use multiprocessing to process the data
with Pool(processes=cpu_count()) as pool:
    results = pool.map(process_item, data_with_indices)

# Update LSH and MinHashes after parallel processing
for min_index, minhash in results:
    lsh.insert(min_index, minhash)
    minhashes[min_index] = minhash


#40 minutes of computation time

#as 10% samle -

In [33]:
#@title Check the results
index_to_check = 35065

same_job_descriptions = set(lsh.query(minhashes[index_to_check]) +result_df.loc[index_to_check,'all_indexes'])

print(same_job_descriptions)

for index in same_job_descriptions:
  print(result_df[result_df['min_index']==index]['job_summary'])

{40736, 25473, 35065, 1131, 40950, 41689}
32    "Is your passion in retail?” We are looking fo...
Name: job_summary, dtype: object
33    "Is your passion in retail?” We are looking fo...
Name: job_summary, dtype: object
30    "Is your passion in retail?” We are looking fo...
Name: job_summary, dtype: object
35065    Job Description:\nKitchen Manager\nEmployer:\n...
Name: job_summary, dtype: object
29    "Is your passion in retail?” We are looking fo...
Name: job_summary, dtype: object
31    "Is your passion in retail?” We are looking fo...
Name: job_summary, dtype: object


In [34]:
#@title Groupung by the non-unique jobs
result_df.sort_values('nbr_of_same_jobs',ascending = False).head(20)

,job_summary,min_index,all_indexes,nbr_of_same_jobs
22073,Dollar General Corporation has been delivering...,23514,"[23514, 23670, 23691, 23794, 23891, 23892, 242...",788
62567,Store Family Dollar\nFamily Dollar is seeking ...,47060,"[47060, 47121, 47165, 47177, 47252, 47289, 473...",601
48410,"Our values start with our people, join a team ...",2997,"[2997, 3164, 3194, 23545, 23652, 25116, 26381,...",406
31616,"Job Description\nDepartment Supervisors lead, ...",31680,"[31680, 52613, 52942, 55068, 55121, 55481, 555...",333
62564,Store Dollar Tree\nWork where you love to shop...,18390,"[18390, 19950, 22163, 22226, 22394, 22487, 224...",286
22105,Dollar General Corporation has been delivering...,13974,"[13974, 23571, 25320, 25376, 26064, 26222, 262...",184
71876,WE ARE LONGHORN.\nLegendary food and service b...,33839,"[33839, 46537, 48620, 49113, 49243, 50645, 506...",161
48413,"Our values start with our people, join a team ...",23661,"[23661, 24864, 25217, 26481, 26483, 26557, 276...",147
62565,Store Dollar Tree\nWork where you love to shop...,8331,"[8331, 16013, 16302, 17944, 22582, 28773, 2879...",147
22109,Dollar General Corporation has been delivering...,12,"[12, 22, 47, 54, 61, 86, 122, 142, 156, 163, 1...",140


In [35]:
#@title Check the results 2

index_to_check = 11939

same_job_descriptions = set(lsh.query(minhashes[index_to_check]) +result_df.loc[index_to_check,'all_indexes'])

for index in same_job_descriptions:
  print(result_df[result_df['min_index']==index]['job_summary'])

22093    Dollar General Corporation has been delivering...
Name: job_summary, dtype: object
22086    Dollar General Corporation has been delivering...
Name: job_summary, dtype: object
22104    Dollar General Corporation has been delivering...
Name: job_summary, dtype: object
22102    Dollar General Corporation has been delivering...
Name: job_summary, dtype: object
22095    Dollar General Corporation has been delivering...
Name: job_summary, dtype: object
22091    Dollar General Corporation has been delivering...
Name: job_summary, dtype: object
22105    Dollar General Corporation has been delivering...
Name: job_summary, dtype: object
22103    Dollar General Corporation has been delivering...
Name: job_summary, dtype: object
22085    Dollar General Corporation has been delivering...
Name: job_summary, dtype: object
22090    Dollar General Corporation has been delivering...
Name: job_summary, dtype: object
22096    Dollar General Corporation has been delivering...
Name: job_summary, dt

In [36]:
#@title Installing packages

!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz
!tar xf spark-3.5.1-bin-hadoop3.tgz
!pip install -q findspark

!pip install pyspark
!pip install findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.1-bin-hadoop3"
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
# Start Spark session
spark = SparkSession.builder.appName("MinHashLSH from Text").getOrCreate()

from pyspark.context import SparkContext
sc = SparkContext.getOrCreate()


# IMPORTING LIBRARIES
import pandas as pd
import numpy as np


from functools import reduce
from pyspark.sql.functions import lit, collect_set, col, broadcast, expr, collect_list, countDistinct, count
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import ArrayType, StructType, StructField, StringType
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import col
from pyspark.ml.feature import MinHashLSH
from pyspark.sql import SparkSession
from pyspark.ml.feature import HashingTF, IDF, CountVectorizer
from pyspark.ml.linalg import Vectors
from pyspark.sql import Row
from pyspark.ml.feature import Tokenizer
from pyspark.sql.functions import udf, col
from pyspark.sql.types import ArrayType, StringType

#Import packages to work with text data

import string
import nltk
nltk.download('punkt_tab')
nltk.download('wordnet')
from nltk.tokenize import (sent_tokenize, word_tokenize)


!pip install unidecode
import unidecode

#Stemmer
#from nltk.stem import PorterStemmer

import re
from IPython.display import clear_output
# Clear the output
clear_output()

In [37]:
#Saving the sample from dataset to upload it to pyshark
result_df[0:100000].to_csv("result.csv", index=False)

In [38]:
#Uploading dataframe
df = spark.read.csv("result.csv", header=True,multiLine=True,
    escape='"',        # Escape character for quotes
    quote='"',         # Quote character to handle multi-line strings
    inferSchema=True   # Infer data types
                    )

In [49]:
#@title Tokenize text by words
tokenizer = Tokenizer(inputCol="job_summary", outputCol="words")
tokenized = tokenizer.transform(df)

hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=250)
featurizedData = hashingTF.transform(tokenized)

countVectorizer = CountVectorizer(inputCol="words", outputCol="rawFeatures")
model = countVectorizer.fit(tokenized)
featurizedData = model.transform(tokenized)

In [50]:
#@title Featurized data and MinHashLSH (Tokenize text by words)
featurizedData.select("min_index", "rawFeatures").show(truncate=False)

idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)

df_new = rescaledData.select("min_index", "features").show(truncate=False)


df_new = rescaledData.select("min_index", "features")
df_new.show(truncate=False)

mh = MinHashLSH()
mh.setInputCol("features")
mh.setOutputCol("hashes")
mh.setSeed(12345)
model = mh.fit(df_new)
model.setInputCol("features")
model.setInputCol("features")


+---------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

MinHashLSHModel: uid=MinHashLSH_7752dc23058e, numHashTables=1

In [51]:
#@title Similarity estimation (Tokenize text by words)

model.approxSimilarityJoin(df_new, df_new, 0.5, distCol="JaccardDistance").select(
    col("datasetA.min_index").alias("min_indexA"),
    col("datasetB.min_index").alias("min_indexB"),
    col("JaccardDistance")).createOrReplaceTempView("similarity_view_words")

In [52]:
#@title Similarity examples (Tokenize text by words)

jacard_dist_approx = 0.5
index = 11939
result = spark.sql(f"SELECT min_indexA,min_indexB FROM similarity_view_words where JaccardDistance < {jacard_dist_approx} and min_indexA = {index} LIMIT 10")
result.show()


+----------+----------+
|min_indexA|min_indexB|
+----------+----------+
|     11939|     25669|
|     11939|     41224|
|     11939|      7040|
|     11939|     25212|
|     11939|     26802|
|     11939|     23678|
|     11939|     42385|
|     11939|       724|
|     11939|     28697|
|     11939|     26856|
+----------+----------+



In [44]:
#@title Tokenize text by sequence of words , creating ngrams, vectorizing

from pyspark.sql.functions import udf

# Tokenize text by sequence of words
tokenizer = Tokenizer(inputCol="job_summary", outputCol="words")
tokenized = tokenizer.transform(df)

# Define a UDF to generate N-grams
k = 3

#Create the function with ngram
def generate_ngrams(words):
    return [" ".join(words[i:i+k]) for i in range(len(words) - k +1 )]

# Register UDF
generate_ngrams_udf = udf(generate_ngrams, ArrayType(StringType()))

# Apply the UDF to create a new column for trigrams
ngram_df = tokenized.withColumn("ngram", generate_ngrams_udf(col("words")))
ngram_df.select("min_index", "ngram").show(truncate=False)

#Hashing ngrams
hashingTF = HashingTF(inputCol="ngram", outputCol="rawFeatures", numFeatures=250, binary = True) #, binary = True
featurizedData = hashingTF.transform(ngram_df)
featurizedData = hashingTF.transform(ngram_df)

#Creating the vectors
countVectorizer = CountVectorizer(inputCol="ngram", outputCol="rawFeatures") #,vocabSize=10000, minDF=2
model = countVectorizer.fit(ngram_df)
featurizedData = model.transform(ngram_df)

+---------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [45]:
#Creating new df before modeling
df_new = featurizedData.select("min_index", "rawFeatures")
df_new.show(truncate=False)


+---------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [46]:
#@title Modeling sequence of words

#Modeling
mh = MinHashLSH()
mh.setInputCol("rawFeatures")
mh.setOutputCol("hashes")

#Set seeds
mh.setSeed(12345)

#Fit modeling
model = mh.fit(df_new)
model.setInputCol("rawFeatures")
model.setInputCol("features")


MinHashLSHModel: uid=MinHashLSH_6a28d6fd4cdb, numHashTables=1

In [47]:
#@title Modeling sequence of words

model.approxSimilarityJoin(df_new, df_new, 0.5, distCol="JaccardDistance").select(
    col("datasetA.min_index").alias("min_indexA"),
    col("datasetB.min_index").alias("min_indexB"),
    col("JaccardDistance")).createOrReplaceTempView("similarity_view_seq")

In [48]:
#@title Examples of pairs

jacard_dist_approx = 0.5
index = 11939
result = spark.sql(f"SELECT min_indexA,min_indexB FROM similarity_view_seq where min_indexA = {index}  Limit 5")
result.show()

+----------+----------+
|min_indexA|min_indexB|
+----------+----------+
|     11939|     26759|
|     11939|     89794|
|     11939|     42420|
|     11939|     25212|
|     11939|     26802|
+----------+----------+

